In [6]:
# imports
import tensorflow as tf
from tensorflow.contrib.layers import xavier_initializer as xav
import numpy as np

In [7]:
class controller(object):
    """
    Basic Feed forward neural network 
    
    """
    def __init__(self, layers):
        self.sess = tf.Session()
        self.layers = layers

        with tf.variable_scope("weights", reuse=None):
            prev_size = layers[0]
            cur_layer = 1
            self.weights = []
            xav_init = xav(dtype=tf.float32)
            for size in layers[1:]:
                self.weights.append(tf.get_variable(name="w" + str(cur_layer), shape=[size, prev_size],
                                                    initializer=xav_init))
                prev_size = size
                cur_layer += 1

        self.optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-2, decay=0.2)

    def train(self, prediction, target):

        loss = tf.reduce_sum(tf.square(prediction - target))
        loss_step = self.optimizer.minimize(loss)

        return loss, loss_step


    def forward(self, input):

        layer = input

        with tf.variable_scope("weights", reuse=True):
            for layer_index in range(len(self.layers) - 1):
                layer = tf.matmul(self.weights[layer_index], layer)
                if layer_index != len(self.layers) - 2:
                    layer = tf.nn.relu(layer)

        return layer

In [8]:
class NTM(object):
    """
    Neural Turing Machine
    
    """
    def __init__(self, layers, mem_size):
        self.controller = controller(layers)
        with tf.variable_scope("ntm", reuse=None):
            self.memory = tf.zeros(shape=[mem_size, mem_size], dtype=tf.float32, name="memory")
            self.read_vector = tf.zeros(mem_size)
            xav_init = xav(dtype=tf.float32)
            self.weights = tf.get_variable("weights", initializer=xav_init, shape=[mem_size, 1])


    def read(self):
        with tf.variable_scope("ntm", reuse=True):
            mem = self.memory
            wts = self.weights
            self.read_vector = tf.matmul(mem, wts)


    def write(self):        
        pass
    
    def copy_train(self):
        pass
    
    def copy(self):
        pass

In [9]:
with tf.Graph().as_default():
    ntm = NTM([100,200,300,200,40], 5)
    ntm.read()
    
    sess = tf.Session()
    
    # writer = tf.summary.FileWriter("/Users/shravan/codes/NTM/logs")
    # writer.add_graph(sess.graph)
       
    sess.run(tf.global_variables_initializer())
    read_vec = sess.run(ntm.read_vector)
    print(read_vec)

[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
